In [1]:
# !pip install opencv-python
# !pip install pydicom
# !pip install python-gdcm
# !pip install pycimg
# !pip install napari
# !pip install nibabel


In [ ]:
import nibabel as nib
import numpy as np
import glob
import pydicom
import matplotlib.pyplot as plt
import os
import cv2

In [35]:
class reading_orignal_data:
    
    def __init__(self, MAIN_PATH,SUB_PATH={"CT_PATH":'/CT/',"RT_PATH":'/RTStruct/',}):

        self.INPUT_DIRECTORY = sorted(glob.glob(MAIN_PATH))
        self.PTV_IMAGES = []
        self.CT_SCAN_IMAGES = []

        for frakDir in self.INPUT_DIRECTORY: 

            ctDir     = frakDir + SUB_PATH["CT_PATH"]

            StructDicomFileName = glob.glob(frakDir+ SUB_PATH["RT_PATH"]+ '/*')[0]
            roiId=next((roi[0] for roi in self.displayStructureIDs(StructDicomFileName) if roi[1].lower() == "ptv"),None)
    
            if roiId !=None:
                _,ct = self.readDICOM3D(ctDir)
                cnt  = self.drawContours(ct.shape,roiId,ctDir,StructDicomFileName)

                cnt[cnt>0] = 1

                self.PTV_IMAGES.append(cnt)
                self.CT_SCAN_IMAGES.append(ct)

    def get_PTV_IMAGES(self):
        return self.PTV_IMAGES
    
    def get_CT_SCAN_IMAGES(self):
        return self.CT_SCAN_IMAGES

    def readDICOM3D(self,DICOM_DIR):
        dicomFiles = [file for file in os.listdir(DICOM_DIR) if file.endswith('dcm') and file.startswith('CT')]
        dicoms = []
        for file in dicomFiles:
            ds = pydicom.dcmread(DICOM_DIR+file)
            dicoms.append((DICOM_DIR+file,int(ds[0x0020,0x0013].value)))
        dicoms = sorted(dicoms, key=lambda x: x[1])

        im3D = []
        slicePositions = []
        for d in dicoms:
            ds = pydicom.dcmread(d[0])
            slicePositions.append(ds.ImagePositionPatient[2])
            im3D.append(ds.pixel_array)
    
        im3D = np.asarray(im3D,dtype=np.int16)
        im3D = np.swapaxes(im3D,0,1)
        im3D = np.swapaxes(im3D,1,2)

        ds = pydicom.dcmread(dicoms[0][0])
        CTOrigin = ds.ImagePositionPatient
        CTPixelSize = ds.PixelSpacing
        CTSliceThickness = ds.SliceThickness

        grid = (np.arange(CTOrigin[1],CTOrigin[1]+CTPixelSize[1]*im3D.shape[0],CTPixelSize[1]),
                np.arange(CTOrigin[0],CTOrigin[0]+CTPixelSize[0]*im3D.shape[1],CTPixelSize[0]),
                np.asarray(slicePositions,dtype=np.float64))
    
        return grid,im3D
    
    def displayStructureIDs(self,structuresFile):
        ROINumbers = []
        ds = pydicom.dcmread(structuresFile)
        for _,struct in enumerate(ds.StructureSetROISequence):
            ROINumbers.append((struct.ROINumber,struct.ROIName))
        return ROINumbers
    
    def drawContours(self,imSize,roiID,pathToCT,structuresDicomFileName):

        ds = pydicom.dcmread(structuresDicomFileName)

        roiNumbers = [dum.ROINumber for dum in ds.StructureSetROISequence]
        structID = roiNumbers.index(roiID)
    

        assert ds.SOPClassUID == "1.2.840.10008.5.1.4.1.1.481.3", "This is not a Dicom Structure file"

        ROI = [ds.ROIContourSequence[u].ContourSequence for u in range(len(ds.ROIContourSequence)) if ds.ROIContourSequence[u].ReferencedROINumber 
               == ds.StructureSetROISequence[structID].ROINumber][0]

        positions = []
        CTs = glob.glob(pathToCT + '/CT*.dcm')
        for fname in CTs:
            ctds = pydicom.dcmread(fname)
            positions.append((ctds.ImagePositionPatient[2],fname)) 
        positions = sorted(positions, key=lambda x: x[0])  
        delta = positions[1][0] - positions[0][0]

        dum = np.zeros(imSize,dtype=np.uint8)
    
        for seq in ROI:
            points = np.swapaxes(np.reshape(seq.ContourData,(-1,3)),0,1)
            pos = points[2,0]
            for p in positions:
                if abs(pos - p[0]) < delta/2:
                    fname = p[1]     
            dicImage = pydicom.dcmread(fname)
        

            M = np.zeros((3,3),dtype = np.float32)
            M[0,0] = dicImage[0x0020, 0x0037].value[1]* dicImage[0x0028, 0x0030].value[0]
            M[1,0] = dicImage[0x0020, 0x0037].value[0]* dicImage[0x0028, 0x0030].value[0]
            M[0,1] = dicImage[0x0020, 0x0037].value[4]* dicImage[0x0028, 0x0030].value[1]
            M[1,1] = dicImage[0x0020, 0x0037].value[3]* dicImage[0x0028, 0x0030].value[1]
            M[0,2] = dicImage[0x0020, 0x0032].value[0]
            M[1,2] = dicImage[0x0020, 0x0032].value[1]
            M[2,2] = 1.0
        
            M = np.linalg.inv(M)
            points[2,:].fill(1)
            points = np.dot(M,points)[:2,:]

            big = int(ds.StructureSetROISequence[structID].ROINumber) # 255
            CTSlice = int(dicImage[0x0020,0x0013].value)-1            # numery sliców w Dicom startują od 1
            dum2D = np.zeros(imSize[0:2],dtype=np.uint8)
            for id in range(points.shape[1]-1):
                cv2.line(dum2D,(int(points[1,id]),int(points[0,id])),(int(points[1,id+1]),int(points[0,id+1])),big,1)
            cv2.line(dum2D,(int(points[1,points.shape[1]-1]),int(points[0,points.shape[1]-1])),(int(points[1,0]),int(points[0,0])),big,1)

            dum[dum2D!=0,CTSlice] = dum2D[dum2D!=0]
        
        for sl in range(dum.shape[2]):
            im_flood_fill = dum[...,sl].copy()
            h, w = dum.shape[:2]
            mask = np.zeros((h + 2, w + 2), np.uint8)
            im_flood_fill = im_flood_fill.astype("uint8")
            cv2.floodFill(im_flood_fill, mask, (0, 0), 128)
            dum[im_flood_fill!=128,sl] = big
        
        return dum
    
    @staticmethod
    def saveImage(PATH,LABEL,IMAGES):
        aff = np.eye(4)
        for INDEX,IMG in enumerate(IMAGES):
            baseName = PATH + LABEL + str(INDEX) + '_.nii.gz'
            niftiImage = nib.Nifti1Image(IMG, affine=aff)
            nib.save(niftiImage,baseName)

In [36]:
orignal_data_ob=reading_orignal_data('samples/Pacjent_01_anonymized/frakcja_*')

In [ ]:
reading_orignal_data.saveImage("Test/","PTV_",orignal_data_ob.get_PTV_IMAGES())
reading_orignal_data.saveImage("Test/","CT_",orignal_data_ob.get_CT_SCAN_IMAGES())